## Code from AutoGPT that has some good ideas

In [ ]:
import os
import requests
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

def download_video(url, save_location, progress_bar):
    try:
        response = requests.get(url, stream=True)
        total = int(response.headers.get('content-length', 0))

        with open(save_location, 'wb') as file, tqdm(
            desc=save_location,
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)

        return True
    except Exception as e:
        print(e)
        return False

def browse_location():
    filename = filedialog.asksaveasfilename(initialdir = "/", title = "Select file", defaultextension=".mp4", filetypes = (("mp4 files","*.mp4"),("mkv files","*.mkv")))
    location_entry.delete(1, tk.END)
    location_entry.insert(tk.END, filename)

def start_download():
    progress_bar['value'] = 0
    root.update_idletasks()

    url = url_entry.get()
    save_location = location_entry.get()
    
    if url=="" or save_location=="":
        messagebox.showerror("Error", "Both URL and Save location are required!")
        return

    if not url.lower().endswith((".mkv", ".mp4")):
        messagebox.showerror("Error", "Please input a direct link to an mkv or mp4 file!")
        return
    
    success = download_video(url, save_location, progress_bar)
    if success:
        messagebox.showinfo("Success", "Video file has been downloaded and saved!")
    else:
        messagebox.showerror("Error", "Could not download file")

root = tk.Tk()
root.geometry("500x150")

url_label = tk.Label(root, text="URL: ")
url_label.grid(row = 0, column = 0)

url_entry = tk.Entry(root, width = 50)
url_entry.grid(row = 0, column = 1)

location_label = tk.Label(root, text="Save As: ")
location_label.grid(row = 1, column = 0)

location_entry = tk.Entry(root, width = 50)
location_entry.grid(row = 1, column = 1)

browse_button = tk.Button(root, text = "Browse", command = browse_location)
browse_button.grid(row = 1, column = 2)

download_button = tk.Button(root, text = "Download", command = start_download)
download_button.grid(row = 2, column = 1)

progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode='determinate')
progress_bar.grid(row = 3, column = 1)

root.mainloop()

In [ ]:
# filename: video_downloader.py

from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QLineEdit, QMessageBox, QWidget
from PyQt5.QtCore import Qt
import sys

class DownloadWindow(QMainWindow):
    def __init__(self, parent=None):
        super(DownloadWindow, self).__init__(parent)

        self.download_button = QPushButton("Download")
        self.download_button.clicked.connect(self.download_vid)

        self.url_input = QLineEdit()

        layout = QVBoxLayout()
        layout.addWidget(self.url_input)
        layout.addWidget(self.download_button)

        main_widget = QWidget()
        main_widget.setLayout(layout)

        self.setCentralWidget(main_widget)

    def download_vid(self):
        url = self.url_input.text()
        if not url:
            QMessageBox.warning(self, "Warning", "Enter a url to download from.")
        else:
            # Write the function to download the video here
            print(f"Downloading from {url}")

def main():
    app = QApplication(sys.argv)

    main_window = DownloadWindow()
    main_window.show()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()

In [ ]:
import os
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import filedialog, messagebox
from urllib.parse import urlparse
from urllib.request import urlretrieve
import logging
from ttkthemes import ThemedTk

logging.basicConfig(filename='error.log', level=logging.ERROR)

def validate_url(url):
    parsed = urlparse(url)
    if not (parsed.path.endswith(".mp4") or parsed.path.endswith(".mkv")):
        return False
    return True

def download_video(url, save_location):
    progress_bar.start()
    try:
        def report_hook(block_num, block_size, total_size):
            percent = int(block_num * block_size * 100 / total_size)
            progress_bar['value'] = percent
            root.update_idletasks()

        urlretrieve(url, save_location, reporthook=report_hook)
        progress_bar.stop()
        return True
    except urllib.error.URLError as e:
        messagebox.showerror("Error", str(e))
        logging.error(str(e))
        progress_bar.stop()
        return False
    except Exception as e:
        messagebox.showerror("Error", str(e))
        logging.error(str(e))
        progress_bar.stop()
        return False

def browse_location():
    filename = filedialog.asksaveasfilename(initialdir = "/", title = "Select file", defaultextension=".mp4", filetypes = (("mp4 files","*.mp4"),("mkv files","*.mkv")))
    location_entry.delete(1, tk.END)
    location_entry.insert(tk.END, filename)

def start_download():
    url = url_entry.get()
    save_location = location_entry.get()

    if url == "" or save_location == "":
        messagebox.showerror("Error", "Both URL and Save location are required!")
        return

    if not validate_url(url):
        messagebox.showerror("Error", "URL must be a direct link to an .mp4 or .mkv file!")
        return

    success = download_video(url, save_location)
    if success:
        messagebox.showinfo("Success", "The video has been downloaded and saved.")
    else:
        return

root = ThemedTk(theme='arc')
root.geometry("500x150")

url_label = ttk.Label(root, text="URL: ")
url_label.grid(row=0, column=0, padx=10, pady=10)

url_entry = ttk.Entry(root, width=50)
url_entry.grid(row=0, column=1, padx=10, pady=10)

location_label = ttk.Label(root, text="Save As: ")
location_label.grid(row=1, column=0, padx=10, pady=10)

location_entry = ttk.Entry(root, width=50)
location_entry.grid(row=1, column=1, padx=10, pady=10)

browse_button = ttk.Button(root, text="Browse", command=browse_location)
browse_button.grid(row=1, column=2, padx=10, pady=10)

download_button = ttk.Button(root, text="Download", command=start_download)
download_button.grid(row=2, column=1, padx=10, pady=10)

root.columnconfigure(1, weight=1)
root.rowconfigure(2, weight=1)

progress_bar = ttk.Progressbar(root, orient="horizontal", mode="determinate")
progress_bar.grid(row=3, columnspan=3, padx=10, pady=10, sticky="ew")

root.mainloop()